# ДЗ №4, Волжина Елена
[Локальное выравнивание](https://compscicenter.ru/learning/assignments/24207/)


В задаче 2 найти оптимальное выравнивание среди выравниваний всех возможных подслов двух последовательностей – локальное выравнивание.

In [1]:
from collections import defaultdict

M = 5     # match
MM = 3    # mismatch
D = 4     # gap
string1, string2 = "GACTTAC", "CGTGAATTCAT"          # from http://vlab.amrita.edu/?sub=3&brch=274&sim=1433&cnt=1

In [2]:
def align(s1, s2):
    assert s1 and s2, "one of strings is empty!"
    
    n, m = len(s1), len(s2)
    sources = defaultdict(lambda: defaultdict(list))
    weights = [[None for _ in range(m + 1)] for _ in range(n + 1)]
    
    # first row
    for n_col in range(m + 1):
        weights[0][n_col] = 0
        sources[0][n_col] = [(0, 0)]
        
    # first column
    for n_row in range(n + 1):
        weights[n_row][0] = 0
        sources[n_row][0] = [(0, 0)]
    
    sources[0][0] = []
    
    def get_ways(n_row, n_col):
        matched = s1[n_row - 1] == s2[n_col - 1]
        return [
            ((n_row - 1, n_col - 1), M if matched else -MM),  # (mis)match
            ((n_row - 1, n_col), -D),   # move down
            ((n_row, n_col - 1), -D),   # move right
            ((0, 0), 0),                # start from scratch
        ]
    
    # dynamically fill the rest
    def process(n_row, n_col):
        for (old_n_row, old_n_col), diff in get_ways(n_row, n_col):
            prev_value = weights[old_n_row][old_n_col]
            new_value = prev_value + diff
            
            if weights[n_row][n_col] is None or weights[n_row][n_col] < new_value:
                weights[n_row][n_col] = new_value
                sources[n_row][n_col] = [(old_n_row, old_n_col)]
            elif weights[n_row][n_col] == new_value:
                sources[n_row][n_col].append((old_n_row, old_n_col))
    
    for k in range(1, min(n, m) + 1):
        process(k, k)
        for n_row in range(k + 1, n + 1):
            process(n_row, k)
        for n_col in range(k + 1, m + 1):
            process(k, n_col)
        
    return weights, sources

In [3]:
alignment_weights, alignment_sources = align(string1, string2)

In [4]:
print("WEIGHTS MATRIX")
print("\n".join(" ".join("{: >3}".format(x) for x in row) 
                for row in alignment_weights))

WEIGHTS MATRIX
  0   0   0   0   0   0   0   0   0   0   0   0
  0   0   5   1   5   1   0   0   0   0   0   0
  0   0   1   2   1  10   6   2   0   0   5   1
  0   5   1   0   0   6   7   3   0   5   1   2
  0   1   2   6   2   2   3  12   8   4   2   6
  0   0   0   7   3   0   0   8  17  13   9   7
  0   0   0   3   4   8   5   4  13  14  18  14
  0   5   1   0   0   4   5   2   9  18  14  15


In [5]:
def restore_paths(sources, n_row, n_col):
    if sources[n_row][n_col]:
        result = []
        for source in sources[n_row][n_col]:
            paths = restore_paths(sources, *source)
            result.extend([p + [(n_row, n_col)] for p in paths])
        return result
    else:
        return [[]]


In [6]:
local_alignment_paths = []
max_weight = max(w for row in alignment_weights for w in row)
argmax = [(n_row, n_col) 
          for n_row, row in enumerate(alignment_weights) 
          for n_col, weight in enumerate(row) if weight == max_weight]
for n_row, n_col in argmax:
    local_alignment_paths.extend(restore_paths(alignment_sources, n_row, n_col))
local_alignment_paths

[[(0, 3), (1, 4), (2, 5), (3, 6), (4, 7), (5, 8), (5, 9), (6, 10)],
 [(0, 3), (1, 4), (2, 5), (3, 6), (4, 7), (5, 8), (6, 8), (7, 9)]]

In [7]:
def print_alignments(paths):
    for path in paths:
        res1, res2 = [], []
        for (r1, c1), (r2, c2) in zip(path, path[1:]):
            if r1 + 1 == r2 and c1 + 1 == c2:
                # mis(match)
                res1.append(string1[r2 - 1])
                res2.append(string2[c2 - 1])
            elif r1 + 1 == r2 and c1 == c2:
                # down
                res1.append(string1[r2 - 1])
                res2.append("-")
            else:
                # right
                res1.append("-")
                res2.append(string2[c2 - 1])
        print(res1, "\n", res2, "\n\n", sep="")

print("LOCAL ALIGNMENTS:")
print_alignments(local_alignment_paths)

LOCAL ALIGNMENTS:
['G', 'A', 'C', 'T', 'T', '-', 'A']
['G', 'A', 'A', 'T', 'T', 'C', 'A']


['G', 'A', 'C', 'T', 'T', 'A', 'C']
['G', 'A', 'A', 'T', 'T', '-', 'C']


